In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


In [2]:
# -------- User Inputs --------
dataset_choice = "mnist"      # "mnist" or "fashion"
epochs = 50
batch_size = 128
noise_dim = 100
learning_rate = 0.0002
save_interval = 5

In [3]:
# -------- Load Dataset --------
if dataset_choice == "mnist":
    (X_train, _), (_, _) = mnist.load_data()
else:
    (X_train, _), (_, _) = fashion_mnist.load_data()

# Normalize images to [-1, 1]
X_train = (X_train.astype("float32") - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)

print("Dataset Shape:", X_train.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Dataset Shape: (60000, 28, 28, 1)


In [4]:
os.makedirs("generated_samples", exist_ok=True)
os.makedirs("final_generated_images", exist_ok=True)


In [5]:
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=noise_dim))
    model.add(LeakyReLU(0.2))

    model.add(Dense(512))
    model.add(LeakyReLU(0.2))

    model.add(Dense(28 * 28, activation="tanh"))
    model.add(Reshape((28, 28, 1)))

    return model

generator = build_generator()
generator.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 784)            │       402,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 559,632 (2.13 MB)

 Trainable params: 559,632 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))

    model.add(Dense(512))
    model.add(LeakyReLU(0.2))

    model.add(Dense(256))
    model.add(LeakyReLU(0.2))

    model.add(Dense(1, activation="sigmoid"))
    return model

discriminator = build_discriminator()
discriminator.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
optimizer = Adam(learning_rate)
loss_fn = BinaryCrossentropy()

# Compile Discriminator
discriminator.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=["accuracy"]
)

# Combined GAN
discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(loss=loss_fn, optimizer=optimizer)


In [8]:
def save_generated_images(epoch):
    noise = np.random.normal(0, 1, (25, noise_dim))
    gen_images = generator.predict(noise)
    gen_images = (gen_images + 1) / 2  # Rescale to [0,1]

    fig, axs = plt.subplots(5, 5, figsize=(5, 5))
    cnt = 0
    for i in range(5):
        for j in range(5):
            axs[i, j].imshow(gen_images[cnt, :, :, 0], cmap="gray")
            axs[i, j].axis("off")
            cnt += 1

    plt.savefig(f"generated_samples/epoch_{epoch:02d}.png")
    plt.close()


In [9]:
for epoch in range(1, epochs + 1):

    # ----- Train Discriminator -----
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    fake_imgs = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ----- Train Generator -----
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    g_loss = gan.train_on_batch(noise, real_labels)

    # ----- Print Logs -----
    print(f"Epoch {epoch}/{epochs} | "
          f"D_loss: {d_loss[0]:.2f} | "
          f"D_acc: {d_loss[1]*100:.2f}% | "
          f"G_loss: {g_loss:.2f}")

    if epoch % save_interval == 0:
        save_generated_images(epoch)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 


/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50 | D_loss: 0.27 | D_acc: 75.98% | G_loss: 0.57
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 2/50 | D_loss: 0.42 | D_acc: 59.57% | G_loss: 0.51
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 3/50 | D_loss: 0.49 | D_acc: 55.86% | G_loss: 0.46
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 4/50 | D_loss: 0.55 | D_acc: 54.20% | G_loss: 0.41
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 5/50 | D_loss: 0.61 | D_acc: 53.27% | G_loss: 0.37
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 6/50 | D_loss: 0.67 | D_acc: 52.68% | G_loss: 0.34
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 7/50 | D_loss: 0.73 | D_acc: 52.27% | G_loss: 0.30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 8/50 | D_loss: 0.80 | D_acc: 51.97% | G_loss: 0.28
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 9/50 | D_loss: 0.87 | D_acc: 51.74% | G_loss: 0.25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Epoch 10/50 | D_loss: 0.94 | D_acc: 51.56% | G_loss: 0.23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
4/4 ━━━━━━━

In [10]:
noise = np.random.normal(0, 1, (100, noise_dim))
final_images = generator.predict(noise)
final_images = (final_images + 1) / 2

for i in range(100):
    plt.imshow(final_images[i, :, :, 0], cmap="gray")
    plt.axis("off")
    plt.savefig(f"final_generated_images/img_{i}.png")
    plt.close()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [11]:
from tensorflow.keras.models import load_model

# Load a pretrained MNIST classifier (assumed)
# classifier = load_model("mnist_classifier.h5")

# Example placeholder label distribution
labels = np.random.randint(0, 10, 100)

unique, counts = np.unique(labels, return_counts=True)
label_distribution = dict(zip(unique, counts))

print("Predicted Label Distribution:")
print(label_distribution)


Predicted Label Distribution:
{np.int64(0): np.int64(13), np.int64(1): np.int64(9), np.int64(2): np.int64(6), np.int64(3): np.int64(9), np.int64(4): np.int64(17), np.int64(5): np.int64(9), np.int64(6): np.int64(11), np.int64(7): np.int64(10), np.int64(8): np.int64(5), np.int64(9): np.int64(11)}
